In [1]:
import os
import random
from Mydataset import MYDataset
import math
from tqdm import tqdm
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import r2_score
from IPython.display import clear_output as clear
import statsmodels.api as sm
from SWNN import SWNN

In [42]:
def select(data, ratio):
    year = 0
    month = 0
    tdf = pd.DataFrame(columns=data.columns)
    positive = pd.DataFrame(columns=data.columns)
    nagative = pd.DataFrame(columns=data.columns)
    index = -1
    data.sort_values(by=['year', 'month'])
    for i in tqdm(range(0,data.shape[0])):
        if not year == data.loc[i]['year'] or not month == data.loc[i]['month']:
            if not index == -1:
                tp = tdf.sample(frac=ratio)
                tn = tdf.append(tp)
                tn.drop_duplicates(keep=False, inplace=True)
                # print(tdf)
                # print(tp)
                # print(tn)
                positive = positive.append(tp)
                nagative = nagative.append(tn)
            tdf = pd.DataFrame(columns=data.columns)
            year = data.loc[i]['year']
            month = data.loc[i]['month']
            index = i
        tdf.loc[i-index] = data.loc[i]
    return positive, nagative



varName = ['fCO2', 'Chl', 'Temp', 'Salt']
varNum = len(varName)
batch_size = 256

dataset = pd.read_csv("D://CO2_data5.csv", encoding="utf-8")
dataset = dataset.dropna()
dataset = dataset[dataset.index % 128 == 0]

df0 = dataset['date'].str.split("/",expand = True)
df0.columns = ['year', 'month', 'date']

dataset['month'] = df0['month']
dataset['year'] = df0['year']

dataset = dataset.reset_index(drop=True)

test_set, train_set = select(dataset, 0.2)
test_set = test_set.reset_index(drop=True)
train_set = train_set.reset_index(drop=True)

mean_li = []
std_li = []

for i in range(0, varNum, 1):
    mean_li.append(train_set[varName[i]].mean())
    std_li.append(train_set[varName[i]].std())

train_set = train_set.copy()
test_set = test_set.copy()

for i in range(0, varNum, 1):
    train_set.loc[:, varName[i]] = (train_set[varName[i]].copy() - mean_li[i] + 1.0) / std_li[i]
    test_set.loc[:, varName[i]] = (test_set[varName[i]].copy() - mean_li[i] + 1.0) / std_li[i]




  0%|          | 0/2673 [00:00<?, ?it/s]D:\Temp\ipykernel_21140\3251160166.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tn = tdf.append(tp)
D:\Temp\ipykernel_21140\3251160166.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  positive = positive.append(tp)
D:\Temp\ipykernel_21140\3251160166.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nagative = nagative.append(tn)
  0%|          | 11/2673 [00:00<00:24, 108.13it/s]D:\Temp\ipykernel_21140\3251160166.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tn = tdf.append(tp)
D:\Temp\ipykernel_21140\3251160166.py:18: FutureWarning: The frame.append method is deprecated and w

In [16]:
test_set

,date,lon,lat,fCO2,Chl,Temp,Salt,month,year
0,1997/9/16,154.25,47.75,-1.261324,2.416598,-1.278405,-0.363104,9,1997
1,1997/9/16,146.75,37.25,-1.365037,1.793121,0.463093,1.128848,9,1997
2,1997/10/16,-178.25,49.75,-0.54409,1.596729,-1.573049,-0.413502,10,1997
3,1997/10/16,142.75,22.75,-0.344278,1.226352,1.19635,1.627369,10,1997
4,1997/11/16,-117.25,31.25,0.337042,1.617689,0.11437,0.48632,11,1997
...,...,...,...,...,...,...,...,...,...
538,2020/10/16,-139.75,37.75,1.767482,1.272051,0.210249,0.376238,10,2020
539,2020/10/16,-169.25,7.75,0.497094,1.203942,1.289508,1.066518,10,2020
540,2020/10/16,-165.25,47.75,0.372176,1.697416,-1.234146,-0.527737,10,2020
541,2020/11/16,171.25,35.75,-0.196985,1.285755,0.23516,1.613064,11,2020


In [17]:
def compute_distances(P, C):
    A = (P**2).sum(axis=1, keepdims=True)
    B = (C**2).sum(axis=1, keepdims=True).T
 
    return np.sqrt(A + B - 2* np.dot(P, C.T))

In [6]:
len(test_li)

1621

In [43]:
def process_df(my_set, varName):
    temp_df = pd.DataFrame()

    dataset = my_set.reset_index(drop=True)

    temp_df['label'] = dataset[varName[0]]
    temp_df['beta'] = np.ones(dataset.shape[0])
    temp_df[varName[1:4]] = dataset[varName[1:4]].astype('float')

    cor_df = pd.DataFrame()

    dataset['lon'][dataset['lon'] < 0] = dataset['lon'][dataset['lon'] < 0].copy() + 360.0
    cor_df['xcor'] = dataset['lon'].astype('float')
    cor_df['ycor'] = dataset['lat'].astype('float')

    sample_pt = np.array([[110.0, 0.0], [290.0,0.0], [110.0, 70.0], [290.0, 70.0]])

    cor_li = cor_df.to_numpy()
    dis_li = compute_distances(cor_li, sample_pt)
    dis_df = pd.DataFrame(dis_li)
    temp_df = temp_df.join(dis_df)

    return temp_df



train_data = MYDataset(process_df(my_set=train_set, varName=varName), len(varName))
test_data = MYDataset(process_df(my_set=test_set, varName=varName), len(varName))
train_loader = DataLoader(train_data, batch_size=128, shuffle=True, num_workers=0, drop_last=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False, num_workers=0)

In [44]:
train_set[varName] = train_set[varName].astype('float')

In [45]:
relation = str()
relation = varName[0]+"~" + "+".join(varName[1:len(varName)])
fit=sm.formula.ols(relation,data=train_set).fit()

In [55]:
def test():
    model.eval()
    criterion1 = nn.MSELoss()
    criterion2 = nn.L1Loss()
    label_li = np.array([])
    out_li = np.array([])
    val_loss1 = 0
    val_loss2 = 0


    with torch.no_grad():
        for data, coef, label in test_loader:
            #data, coef, label = data.cuda(), coef.cuda(), label.cuda()
            data,label = data.view(data.shape[0], -1), label.view(data.shape[0], -1)

            output = model(data).mul(coef)
            output = out(output)

            loss1 = criterion1(output, label)
            loss2 = criterion2(output, label)

            a = output.view(-1).cpu().detach().numpy()
            b = label.view(-1).cpu().numpy()
            out_li = np.append(out_li, a)
            label_li = np.append(label_li, b)
            

            val_loss1 += loss1.item()*data.size(0)
            val_loss2 += loss2.item()*data.size(0)

        val_loss1 = val_loss1/len(test_loader.dataset)
        val_loss2 = val_loss2/len(test_loader.dataset)
        r2 = r2_score(label_li, out_li)

        diff = out_li - label_li
        diff = np.abs(diff) / np.abs(label_li) * 100

        rmse = math.sqrt(val_loss1) * std_li[0]
        mae  = val_loss2*std_li[0]
        mape = diff.mean()

    print(diff, rmse, mae, mape)

In [47]:
model = SWNN(outsize=4)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [27]:
varName[0]+"+" + "~".join(varName[1:len(varName)])

'fCO2+Chl~Temp~Salt'

In [48]:
r2 = 0
weightlist = [[]]
temp = []
for j in fit.params:
    weightlist[0].append(j)
out = nn.Linear(4, 1, bias = False)
out.weight = nn.Parameter(torch.tensor(weightlist), requires_grad=False)

def train(epoch):
    model.train()
    train_loss = 0
    global r2
    global out
    for data, coef, label in train_loader:
        data = data.view(data.shape[0], -1)
        label = label.view(data.shape[0], -1)
        optimizer.zero_grad()

        output = model(data)
        output = output.mul(coef)
        output = out(output)

        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        a = output.view(-1).detach().numpy()
        b = label.view(-1).numpy()
        if epoch % 100 == 0:
            r2 = r2_score(a, b)

        train_loss += loss.item()*data.size(0)
        
    train_loss = train_loss/len(train_loader.dataset)
    print('\r Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))

def val(epoch):
    model.eval()
    global out
    global r2
    val_loss = 0

    label_li = np.array([])
    out_li = np.array([])

    with torch.no_grad():
        for data, coef, label in test_loader:
            data = data.view(data.shape[0], -1)
            label = label.view(data.shape[0], -1)

            output = model(data)
            output = output.mul(coef)
            output = out(output)

            loss = criterion(output, label)

            a = output.view(-1).detach().numpy()
            b = label.view(-1).numpy()
            out_li = np.append(out_li, a)
            label_li = np.append(label_li, b)

            val_loss += loss.item()*data.size(0)
        val_loss = val_loss/len(test_loader.dataset)
        label_li = np.array(label_li).reshape(-1)
        out_li = np.array(out_li).reshape(-1)
        if epoch % 100 == 0:
            r2 = r2_score(label_li, out_li)
        print('\r Epoch: {} \tTraining Loss: {:.6f} \tR2: {:.6f}'.format(epoch, val_loss, r2))
        if epoch % 2 == 0:
            clear()
        

In [57]:
for epoch in tqdm(range(1, 1000+1)):
    train(epoch=epoch)
    val(epoch=epoch)

 79%|███████▉  | 794/1000 [02:22<00:44,  4.59it/s]

 Epoch: 795 	Training Loss: 0.452844


 80%|███████▉  | 795/1000 [02:22<00:36,  5.58it/s]

 Epoch: 795 	Training Loss: 0.590781 	R2: 0.247249
 Epoch: 796 	Training Loss: 0.444680


KeyboardInterrupt: 

In [60]:
test()

[7.32251518e+01 7.68712697e+02 2.16962462e+02 1.03026928e+02
 1.12976830e+02 4.05062396e+01 8.35752615e+01 5.02872083e+01
 5.48823991e+01 5.25887697e+01 3.69779850e+01 1.73886678e+02
 3.14811269e+02 8.87705991e+01 1.93883603e+01 1.26409002e+02
 6.69773963e+01 3.56222554e+01 3.64967596e+01 8.30579854e+01
 3.64593915e+02 2.70428825e+02 2.18008010e+02 3.22702798e+02
 6.44047621e+01 5.30949121e+01 1.30135154e+02 6.15311053e+01
 2.49265318e+02 5.07852878e+01 1.10213457e+02 4.31870854e+02
 5.41639682e+01 8.28344200e+01 1.00077359e+02 5.76554861e+01
 9.69710730e+01 5.54101178e+01 1.13921935e+02 9.83874233e+01
 6.19528298e+01 9.83603720e+01 4.92608574e+01 2.51830833e+02
 1.78355983e+02 1.75112013e+02 1.27653158e+02 1.03068872e+02
 3.01296151e+01 5.87015828e+02 7.65252937e+00 2.22242146e+01
 6.76005604e+01 1.59368242e+02 8.52690313e+02 7.16844875e+01
 1.97150023e+01 5.21416177e+01 7.56893628e+01 1.73598418e+03
 4.58878462e+01 3.23785240e+01 3.18390548e+03 3.68760882e+01
 7.72419227e+01 1.169416